# Importing libraries

In [1]:
import os
import sys
import glob
import torch
import torchio
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import ShuffleSplit

In [2]:
import imp

import utils.data_processor as data_processor
imp.reload(data_processor)
from utils.data_processor import *

import utils.visualization_tools as visualization_tools
imp.reload(visualization_tools)
from utils.visualization_tools import *

import utils.routine_pirogov as routine
imp.reload(routine)
from utils.routine_pirogov import *

import utils.metrics as metrics
imp.reload(metrics)
from utils.metrics import *

import utils.metrics_deep_mind as metrics_deep_mind
imp.reload(metrics_deep_mind)
from utils.metrics_deep_mind import *

In [3]:
def get_brain_borders(brain):
    
    '''
    Function finds borders of non-zero pixels for a given brain.

    Arguments:
        * brain (np.array): brain
        
    Output:
        * [(min(axial_coord), max(axial_coord)),
        (min(coronal_coord), max(coronal_coord)),
        (min(sagital_coord), max(sagital_coord))] : corresponding borders 
    '''
    
    if isinstance(img, torch.Tensor):
        brain = brain.numpy()
        if (len(brain.shape) > 3):
            brain = brain[0,:,:,:]
                
    elif isinstance(brain, nibabel.nifti1.Nifti1Image):    
        brain = brain.get_fdata()
    
    axial_coord = []
    for i in range(brain.shape[2]):
        if (brain[:, :, i] != 0).any():
            axial_coord.append(i)
    
    coronal_coord = []
    for i in range(brain.shape[1]):
        if (brain[:, i, :] != 0).any():
            coronal_coord.append(i)
    
    sagital_coord = []
    for i in range(brain.shape[0]):
        if (brain[i, :, :] != 0).any():
            sagital_coord.append(i)
                        
    return [(min(axial_coord), max(axial_coord)), (min(coronal_coord), max(coronal_coord)), (min(sagital_coord), max(sagital_coord))]

In [5]:
def get_dataset_borders(dataset_path = '../datasets/new_dataset/fcd_brains/'):
    
    '''
    Function finds borders of non-zero pixels for dataset of brains.

    Arguments:
        * dataset_path (str): path to dataset
    '''
    
    sagital_min, sagital_max, coronal_min, coronal_max, axial_min, axial_max = [], [], [], [], [], []

    for brain in tqdm(os.listdir(dataset_path)):
        coords =  get_brain_borders(load_nii_to_array(dataset_path + brain))
        sagital_min.append(coords[0][0])
        sagital_max.append(coords[0][1])
        coronal_min.append(coords[1][0])
        coronal_max.append(coords[1][1])
        axial_min.append(coords[2][0])
        axial_max.append(coords[2][1])
        
    return np.min(sagital_min), np.max(sagital_max), np.min(coronal_min), np.max(coronal_max), np.min(axial_min), np.max(axial_max)

# Crope brains

In [6]:
def crope_brains(dataset_path='../datasets/new_dataset/fcd_brains/',
                 new_dataset_path='../datasets/croped_new_dataset/fcd_brains/'):
    
    '''
    Function makes croped datset, based on results of get_dataset_borders function

    Arguments:
        * dataset_path (str): path to datset
        * new_dataset_path (str): path to croped dataset
    '''
    
    sagital_min, sagital_max, coronal_min, coronal_max, axial_min, axial_max = get_dataset_borders(dataset_path)
    for brain in tqdm(os.listdir(dataset_path)):
        
        array_brain = load_nii_to_array(dataset_path + '/' + brain)
        array_brain = array_brain[sagital_min: sagital_max, coronal_min: coronal_max, axial_min: axial_max]

        affine = nib.load(dataset_path + '/' + brain).affine
        new_image = nib.Nifti1Image(array_brain.astype(float), affine)
        
        nib.save(new_image, new_dataset_path + brain)

In [7]:
crope_brains(dataset_path='../datasets/new_dataset/fcd_brains/',
             new_dataset_path='../datasets/croped_new_dataset/fcd_brains/')

100%|██████████| 122/122 [07:51<00:00,  3.87s/it]


# Crope masks

In [11]:
def crope_masks(mask_path = '../datasets/new_dataset/masks/',
                dataset_path = '../datasets/new_dataset/fcd_brains/',
                new_mask_path = '../datasets/croped_new_dataset/masks/',
                new_datset_path = '../datasets/croped_new_dataset/fcd_brains/'):
    '''
    Function makes croped masks, based on results of get_dataset_borders

    Arguments:
        * folder_path (str): path to folder with .nii.gz/.nii files 
        * new_folder_path (str): path to new folder 
    '''
    
    sagital_min, sagital_max, coronal_min, coronal_max, axial_min, axial_max = get_dataset_borders(dataset_path)
    for mask in tqdm(os.listdir(mask_path)):
        array_mask = load_nii_to_array(mask_path + mask)
        array_mask = array_mask[sagital_min: sagital_max, coronal_min: coronal_max, axial_min: axial_max]
        array_mask[array_mask > 0] = 1.

        name = mask[5:].split('.')[0]
        affine = nib.load(new_datset_path + name + '.1.nii.gz').affine
        new_mask = nib.Nifti1Image(array_mask.astype(float), affine)
        
        nib.save(new_mask, new_mask_path + mask)

In [12]:
crope_masks(mask_path = '../datasets/new_dataset/masks/',
                dataset_path = '../datasets/new_dataset/fcd_brains/',
                new_mask_path = '../datasets/croped_new_dataset/masks/',
                new_datset_path = '../datasets/croped_new_dataset/fcd_brains/')

100%|██████████| 122/122 [03:02<00:00,  1.50s/it]


# Making CSV file with paths

In [13]:
def make_csv(dataset_path='../datasets/croped_new_dataset/fcd_brains/',
            mask_path='../datasets/croped_new_dataset/masks/',
            name='Pirogov_paths.csv'):
    
    '''
    Function makes csv file with all the paths 

    Arguments:
        * folder_path (str): path to folder with .nii.gz/.nii files 
        * new_folder_path (str): path to new folder 
    '''
    
    subjects = set([x.split('.')[0] for x in os.listdir(dataset_path) if x[:3] == 'fcd'])
    paths_to_all_files = pd.DataFrame(columns = ['Subject', 'T1', 'T2', 'T3', 'fcd_mask'])

    for i, subject in enumerate(subjects):

        path_to_mask = mask_path + 'mask_' + subject + '.1.nii.gz'
        path_to_T1_file = dataset_path + subject + '.1.nii.gz'
        path_to_T2_file = dataset_path + subject + '.2.nii.gz'
        path_to_T3_file = dataset_path + subject + '.3.nii.gz'

        paths_to_all_files.loc[i] = [subject, 
                                      path_to_T1_file, 
                                      path_to_T2_file, 
                                      path_to_T3_file,
                                      path_to_mask]

    paths_to_all_files.to_csv(name)

In [14]:
make_csv(dataset_path='../datasets/croped_new_dataset/fcd_brains/',
            mask_path='../datasets/croped_new_dataset/masks/',
            name='Pirogov_paths.csv')

# Making landmarks

In [15]:
paths_to_all_files = pd.read_csv('Pirogov_paths.csv', index_col = 0)

In [17]:
t1_landmarks_path = 'landmarks/Pirogov_t1_landmarks.npy'
t2_landmarks_path = 'landmarks/Pirogov_t2_landmarks.npy'
t3_landmarks_path = 'landmarks/Pirogov_t3_landmarks.npy'

t1_paths = paths_to_all_files.T1.values.tolist()
t2_paths = paths_to_all_files.T2.values.tolist()
t3_paths = paths_to_all_files.T3.values.tolist()

t1_landmarks = (
    #t1_landmarks_path
    #if t1_landmarks_path.is_file()
    #else 
    HistogramStandardization.train(t1_paths)
)
np.save(t1_landmarks_path, t1_landmarks, allow_pickle=True)

t2_landmarks = (
    #t2_landmarks_path
    #if t2_landmarks_path.is_file()
    #else
    HistogramStandardization.train(t2_paths)
)
np.save(t2_landmarks_path, t2_landmarks, allow_pickle=True)

t3_landmarks = (
    #t2_landmarks_path
    #if t2_landmarks_path.is_file()
    #else
    HistogramStandardization.train(t3_paths)
)
np.save(t3_landmarks_path, t3_landmarks, allow_pickle=True)

100%|██████████| 26/26 [00:56<00:00,  2.19s/it]
